In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [ ]:
data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_6/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_6/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.736842,0.157895,0.736842,0.684211,0.052632]], axis=0)
y = np.append(y, [-0.74650183], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.368421, 0.263158, 0.526316, 0.842105, 0.105263]], axis=0)
y = np.append(y, [-0.32481759704902613], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.359946, 0.009339, 0.744659, 0.973816, 0.016177]], axis=0)
y = np.append(y, [-0.8381637411364704], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.412263, 0.328525, 0.520517, 0.793617, 0.125967]], axis=0)
y = np.append(y, [-0.12430862230397105], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.403200, 0.358308, 0.537100, 0.806405, 0.125919]], axis=0)
y = np.append(y, [-0.2224297640161302], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.391891, 0.368600, 0.531377, 0.802436, 0.120967]], axis=0)
y = np.append(y, [-0.2072371147670746], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.431450, 0.341100, 0.525399, 0.767641, 0.116168]], axis=0)
y = np.append(y, [-0.15947669058205446], axis=0)

#add the week 8 input and result to the array
X = np.append(X, [[0.409767, 0.323588, 0.540202, 0.774776, 0.098109]], axis=0)
y = np.append(y, [-0.28274379072022265], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
def ucb(x, gp, kappa=4.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
"""grid_size = 20
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
x5 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d, e] for a in x1 for b in x2 for c in x3 for d in x4 for e in x5])"""

In [ ]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 5)

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=20, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 5)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)